# Question answering on the SQuAD dataset

## Colab requirements

Before restarting runtime (remeber to select GPU runtime)$\dots$

In [ ]:
!git clone https://github.com/Wadaboa/squad-question-answering.git
!pip install -r squad-question-answering/init/base_requirements.txt

After restarting runtime$\dots$

In [ ]:
import os, sys

sys.path.insert(0, "/content/squad-question-answering")
os.chdir("/content/squad-question-answering")

## Imports

In [1]:
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import transformers

from transformers.trainer_utils import set_seed
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.normalizers import Sequence, StripAccents, Lowercase, Strip
from tokenizers.pre_tokenizers import Sequence as PreSequence
from tokenizers.pre_tokenizers import Whitespace, Punctuation
from tokenizers import BertWordPieceTokenizer

import dataset
import model
import training
import utils

%load_ext autoreload
%autoreload 2

## Initialization

### Matplotlib

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.xmargin'] = .05
plt.rcParams['axes.ymargin'] = .05
plt.style.use('ggplot')

### Weights & biases

In [34]:
%env WANDB_PROJECT=squad-qa
%env WANDB_ENTITY=wadaboa
%env WANDB_MODE=online
%env WANDB_RESUME=never
%env WANDB_WATCH=false
%env WANDB_SILENT=true

env: WANDB_PROJECT=squad-qa
env: WANDB_ENTITY=wadaboa
env: WANDB_MODE=online
env: WANDB_RESUME=never
env: WANDB_WATCH=false
env: WANDB_SILENT=true


In [35]:
!wandb login

In [36]:
!wandb enabled

W&B enabled.


### PyTorch and numpy

In [6]:
RANDOM_SEED = 42
set_seed(RANDOM_SEED)

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

## Preliminaries

### Raw data loading

In [8]:
squad_dataset = dataset.SquadDataset(subset=0.01)

In [9]:
squad_dataset.raw_train_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,56de4d9ecffd8e1900b4b7e2,What year was the Banská Akadémia founded?,Institute_of_technology,1860,The world's first institution of technology or...,1735,167,171
1,572674a05951b619008f7319,What is another speed that can also be reporte...,Film_speed,9354,The standard specifies how speed ratings shoul...,SOS-based speed,793,808
2,5730bb058ab72b1400f9c72c,Where were the use of advanced materials and t...,Sumer,17505,The most impressive and famous of Sumerian bui...,Sumerian temples and palaces,421,449
3,572781a5f1498d1400e8fa1f,Who is elected every even numbered year?,"Ann_Arbor,_Michigan",10585,Ann Arbor has a council-manager form of govern...,mayor,192,197
4,572843ce4b864d190016485c,What was the purpose of top secret ICBM commit...,John_von_Neumann,11497,"Shortly before his death, when he was already ...",decide on the feasibility of building an ICBM ...,194,284
...,...,...,...,...,...,...,...,...
870,5725cad2ec44d21400f3d5a0,It was an early backer of what,Bill_%26_Melinda_Gates_Foundation,8521,A key aspect of the Gates Foundation's U.S. ef...,The foundation was the biggest early backer of...,449,539
871,570c0fedec8fbc190045bc47,What event in February did Barcelona qualify f...,FC_Barcelona,6319,"On 4 January 2016, Barcelona's transfer ban en...",Copa del Rey final,267,285
872,57327ed206a3a419008aca8d,What caused a large migration of Greek refuges...,Humanism,18662,The humanists' close study of Latin literary t...,Greek manuscripts,1262,1279
873,5727c05eff5b5019007d9449,What league are the Toronto Bluejays in?,Exhibition_game,13088,Several MLB teams used to play regular exhibit...,American League,246,261


In [10]:
squad_dataset.raw_test_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,57379ed81c456719005744d7,In what way do idea strings transmit tesion fo...,Force,2058,Tension forces can be modeled using ideal stri...,instantaneously in action-reaction pairs,250,290
1,56e0ed557aa994140058e7dd,What was Tesla's device called?,Nikola_Tesla,181,Tesla also explained the principles of the rot...,Egg of Columbus,187,202
2,56e1febfe3433e140042323a,What is the example of another problem charact...,Computational_complexity_theory,282,What intractability means in practice is open ...,NP-complete Boolean satisfiability problem,539,581
3,57269bb8708984140094cb98,What are EU Regulations essentially the same a...,European_Union_law,759,Although it is generally accepted that EU law ...,Treaty provisions,1332,1349
4,56f884cba6d7ea1400e17708,What theologian differed in views about the so...,Martin_Luther,412,The Lutheran theologian Franz Pieper observed ...,Gerhard,181,188
...,...,...,...,...,...,...,...,...
177,5711475ca58dae1900cd6d8b,"In a 4-cylinder compound engine, what degree w...",Steam_engine,587,With two-cylinder compounds used in railway wo...,180,313,316
178,572fdd03a23a5019007fcaa0,What party had a victory in the 2015 UK election?,Scottish_Parliament,1846,A procedural consequence of the establishment ...,Conservative,465,477
179,572ff56304bcaa1900d76f30,What does the Nederrijn change it's name to?,Rhine,1781,The other third of the water flows through the...,the Lek,402,409
180,5737a0acc3c5551400e51f4a,What may a force on one part of an object affect?,Force,2059,Newton's laws and Newtonian mechanics in gener...,other parts,276,287


### Embeddings

In [11]:
UNK_TOKEN = "[UNK]"
PAD_TOKEN = "[PAD]"

- FastText: 
    - _fasttext-wiki-news-subwords_ (dimensions: 300)
- GloVe:
    - _glove-twitter_ (dimensions: 25. 50, 100, 200)
    - _glove-wiki-gigaword_ (dimensions: 50, 100, 200, 300)
- Word2Vec:
    - _word2vec-google-news_ (dimensions: 300)
    - _word2vec-ruscorpora_ (dimensions: 300)

In [12]:
# See https://github.com/RaRe-Technologies/gensim-data
GLOVE_EMBEDDING_DIMENSION = 25
GLOVE_MODEL_NAME = "glove-twitter"
glove_embedding_model = utils.load_embedding_model(
    GLOVE_MODEL_NAME, embedding_dimension=GLOVE_EMBEDDING_DIMENSION
)

In [13]:
glove_unk = np.mean(glove_embedding_model.vectors, axis=0)
glove_embedding_model.add(UNK_TOKEN, glove_unk)

In [14]:
glove_embedding_model[UNK_TOKEN]

array([-0.459986  ,  0.3140484 ,  0.05261543,  0.02956172, -0.1543944 ,
       -0.37464437, -0.51207143, -0.9670207 ,  0.30385152, -0.43003932,
       -0.25780797,  0.28663048,  0.92323375, -0.02501891,  0.19632967,
        0.3715336 ,  0.27371258,  0.08516268,  0.6888611 ,  0.25681776,
        0.09314184,  0.21430919,  0.05849701, -0.02958639, -0.31859428],
      dtype=float32)

In [15]:
list(glove_embedding_model.vocab.keys())[-1]

'[UNK]'

In [16]:
any(np.all(glove_embedding_model.vectors == 0, axis=1))

False

In [17]:
glove_embedding_model.add(PAD_TOKEN, np.zeros((1, GLOVE_EMBEDDING_DIMENSION)))
glove_embedding_model[PAD_TOKEN]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [18]:
list(glove_embedding_model.vocab.keys())[-1]

'[PAD]'

In [19]:
glove_embedding_model.vectors.shape

(1193516, 25)

In [20]:
glove_vocab = dict(
    zip(glove_embedding_model.index2word, range(len(glove_embedding_model.index2word)))
)

In [21]:
glove_embedding_layer = nn.Embedding(
    glove_embedding_model.vectors.shape[0],
    GLOVE_EMBEDDING_DIMENSION,
    padding_idx=glove_vocab[PAD_TOKEN],
)
glove_embedding_layer.weight = nn.Parameter(
    torch.from_numpy(glove_embedding_model.vectors)
)
glove_embedding_layer.weight.requires_grad = False

### Standard tokenizer and preprocessing

In [22]:
MAX_CONTEXT_TOKENS = 300

In [23]:
standard_question_tokenizer = Tokenizer(WordLevel(glove_vocab, unk_token=UNK_TOKEN))
standard_question_tokenizer.normalizer = Sequence(
    [StripAccents(), Lowercase(), Strip()]
)
standard_question_tokenizer.pre_tokenizer = PreSequence([Whitespace(), Punctuation()])
standard_question_tokenizer.enable_padding(
    direction="right", pad_id=glove_vocab[PAD_TOKEN], pad_type_id=1, pad_token=PAD_TOKEN
)

standard_context_tokenizer = Tokenizer(WordLevel(glove_vocab, unk_token=UNK_TOKEN))
standard_context_tokenizer.normalizer = Sequence([StripAccents(), Lowercase(), Strip()])
standard_context_tokenizer.pre_tokenizer = PreSequence([Whitespace(), Punctuation()])
standard_context_tokenizer.enable_padding(
    direction="right",
    pad_id=glove_vocab[PAD_TOKEN],
    pad_type_id=1,
    pad_token=PAD_TOKEN,
    length=MAX_CONTEXT_TOKENS,
)
standard_context_tokenizer.enable_truncation(MAX_CONTEXT_TOKENS)

In [24]:
standard_tokenizer = dataset.StandardSquadTokenizer(
    standard_question_tokenizer, standard_context_tokenizer, device=DEVICE
)

In [25]:
standard_dm = dataset.SquadDataManager(squad_dataset, standard_tokenizer, device=DEVICE)

In [26]:
standard_dm.tokenizer = standard_tokenizer

In [27]:
standard_dm.train_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,56cd788162d2951400fa65e7,What was Apple's highest quarterly profit as o...,IPod,415,"On January 22, 2008, Apple reported the best q...",[$1.58 billion],[184],[197]
1,56cda41362d2951400fa679b,How many people would be in the orchestra Kond...,The_Legend_of_Zelda:_Twilight_Princess,453,Media requests at the trade show prompted Kond...,[50],[217],[219]
2,56cda8a662d2951400fa67d6,Where did Twilight Princess place among Wii ti...,The_Legend_of_Zelda:_Twilight_Princess,460,Twilight Princess received the awards for Best...,[4th],[777],[780]
3,56cdcfee62d2951400fa687a,Who performs research for Bond?,Spectre_(2015_film),467,Bond disobeys M's order and travels to Rome to...,[Moneypenny],[388],[398]
4,56cdd28562d2951400fa68bd,Who does M fight with?,Spectre_(2015_film),470,Bond and Swann return to London where they mee...,[C],[105],[106]
...,...,...,...,...,...,...,...,...
692,5734580c879d6814001ca544,What official is in charge of Richmond's execu...,"Richmond,_Virginia",17037,Richmond city government consists of a city co...,[mayor],[183],[188]
693,57359c97e853931400426a3f,What is the English translation of Mandap?,Kathmandu,18835,The city of Kathmandu is named after Kasthaman...,[covered shelter],[155],[170]
694,57359f64e853931400426a7e,What notable Nepali figure died in a Kathmandu...,Kathmandu,18841,The Licchavi era was followed by the Malla era...,[Abhaya Malla],[427],[439]
695,5735a721e853931400426aa9,"What are Madhyapur Thimi, Kirtipur and Bhaktapur?",Kathmandu,18846,The agglomeration of Kathmandu has not yet bee...,[municipalities],[441],[455]


In [28]:
standard_dm.val_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,56be8e353aeaaa14008c90c6,"""Charlie's Angels"" featured which single from ...",Beyoncé,63,"The remaining band members recorded ""Independe...",[Independent Women Part I],[37],[61]
1,56be9eea3aeaaa14008c9184,How many awards did Beyonce take home with her...,Beyoncé,79,At the 57th Annual Grammy Awards in February 2...,[three],[108],[113]
2,56bf89cfa10cfb1400551163,What album caused a lawsuit to be filed in 2001?,Beyoncé,63,"The remaining band members recorded ""Independe...",[Survivor],[593],[601]
3,56bf9c70a10cfb14005511bb,In what year did Beyonce have her hiatus?,Beyoncé,72,Beyoncé announced a hiatus from her music care...,[2010],[60],[64]
4,56cc27346d243a140015eebb,During what years did the Mongol leader Kublai...,Sino-Tibetan_relations_during_the_Ming_dynasty,300,Some scholars note that Tibetan leaders during...,[1402–1424],[739],[748]
...,...,...,...,...,...,...,...,...
167,57351496879d6814001cab11,Where can safari hunters go which are uninviti...,Hunting,18825,A variety of industries benefit from hunting a...,[remote areas],[367],[379]
168,5735fcb96c16ec1900b928c8,What forbid hunting in the woods with hounds a...,Hunting,18806,"From early Christian times, hunting has been f...",[Corpus Juris Canonici],[98],[119]
169,5735ffb96c16ec1900b928e1,Game animals were introduced here by whom?,Hunting,18808,New Zealand has a strong hunting culture. The ...,[acclimatisation societies],[189],[214]
170,573605726c16ec1900b92906,Why are assistants used?,Hunting,18812,"Shooting as practised in Britain, as opposed t...",[help load shotguns],[305],[323]


In [29]:
standard_dm.test_df

,question_id,question,title,context_id,context,answer,answer_start,answer_end
0,56be5438acb8001400a5031c,Which California venue was one of three consid...,Super_Bowl_50,5,The league eventually narrowed the bids to thr...,[San Francisco Bay Area's Levi's Stadium],[128],[167]
1,56be5523acb8001400a5032e,When did Lev's Stadium open?,Super_Bowl_50,7,"On May 21, 2013, NFL owners at their spring me...",[2014],[144],[148]
2,56beab833aeaaa14008c91d2,Which Denver linebacker was named Super Bowl MVP?,Super_Bowl_50,2,The Broncos took an early lead in Super Bowl 5...,[linebacker Von Miller],[237],[258]
3,56beb6533aeaaa14008c9290,Who was first on the team in total tackles?,Super_Bowl_50,14,The Broncos' defense ranked first in the NFL y...,[Brandon Marshall],[458],[474]
4,56bebc383aeaaa14008c9321,What yard marker on the field was painted gold?,Super_Bowl_50,25,Various gold-themed promotions and initiatives...,[50],[232],[234]
...,...,...,...,...,...,...,...,...
175,57379829c3c5551400e51f3d,What does the W and Z boson exchange create?,Force,2056,The weak force is due to the exchange of the h...,[weak force],[4],[14]
176,57379a4b1c456719005744d0,What is the force that causes rigid strength i...,Force,2057,The normal force is due to repulsive forces of...,[normal],[298],[304]
177,57379ed81c456719005744d7,In what way do idea strings transmit tesion fo...,Force,2058,Tension forces can be modeled using ideal stri...,[instantaneously in action-reaction pairs],[250],[290]
178,5737a0acc3c5551400e51f4a,What may a force on one part of an object affect?,Force,2059,Newton's laws and Newtonian mechanics in gener...,[other parts],[276],[287]


## Utils

In [30]:
TRAINER_ARGS = partial(
    transformers.TrainingArguments,
    output_dir="./checkpoints",
    logging_dir="./runs",
    logging_first_step=True,
    logging_steps=5,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    num_train_epochs=5,
    remove_unused_columns=False,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    label_names=["answers"],
    seed=RANDOM_SEED
)

## Baseline model

In [31]:
baseline_model = model.QABaselineModel(glove_embedding_layer, MAX_CONTEXT_TOKENS, device=DEVICE)
print(f"The baseline model has {baseline_model.count_parameters()} parameters")

The baseline model has 41000 parameters


In [32]:
%env WANDB_RUN_GROUP=baseline
baseline_run_name=utils.get_run_name()
baseline_output_dir=f"./checkpoints/baseline/{baseline_run_name}"
baseline_args = TRAINER_ARGS(output_dir=baseline_output_dir,run_name=baseline_run_name)
baseline_trainer = training.SquadTrainer(
    model=baseline_model,
    args=baseline_args,
    data_collator=standard_dm.tokenizer,
    train_dataset=standard_dm.train_dataset,
    eval_dataset=standard_dm.val_dataset,
)

env: WANDB_RUN_GROUP=baseline


In [33]:
baseline_trainer.train()
#wandb.save(f"{baseline_output_dir}/**/*");

  0%|          | 0/55 [00:00<?, ?it/s]Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: wadaboa (use `wandb login --relogin` to force relogin)


 20%|██        | 11/55 [00:08<00:22,  1.98it/s]{'f1': 0.0007833920877399139, 'accuracy': 0.008771929824561403, 'em': 0.0, 'epoch': 0.91}

 67%|██████▋   | 2/3 [00:00<00:00,  4.08it/s]

100%|██████████| 3/3 [00:00<00:00,  3.29it/s]
                                             wandb: WARNING Symlinked 5 files into the W&B run directory, call wandb.save again to sync new files.
{'eval_loss': 15.100722312927246, 'eval_f1': 0.00016189088554314392, 'eval_accuracy': 0.005813953488372093, 'eval_em': 0.0, 'eval_runtime': 1.3401, 'eval_samples_per_second': 128.347, 'epoch': 1.0}
 67%|██████▋   | 2/3 [00:00<00:00,  3.88it/s]

100%|██████████| 3/3 [00:00<00:00,  3.34it/s]
                                             wandb: WARNING Symlinked 5 files into the W&B run directory, call wandb.save again to sync new files.
{'eval_loss': 14.998549461364746, 'eval_f1': 0.00016189088554314392, 'eval_accuracy': 0.005813953488372093, 'eval_em': 0.0, 'eval_runtime': 1.348, 'eval_samples_per_second': 127.597, '

TrainOutput(global_step=55, training_loss=14.79012652310458, metrics={'train_runtime': 37.9985, 'train_samples_per_second': 1.447, 'epoch': 5.0})

In [37]:
baseline_test_output = baseline_trainer.predict(standard_dm.test_dataset)
baseline_test_output.metrics

100%|██████████| 3/3 [00:00<00:00,  3.14it/s]

{'test_loss': 15.37047004699707,
 'test_f1': 0.00017915642915642917,
 'test_accuracy': 0.005555555555555556,
 'test_em': 0.0,
 'test_runtime': 1.3612,
 'test_samples_per_second': 132.237}

In [38]:
baseline_answers_path = "results/baseline.json"
utils.save_answers(baseline_answers_path, baseline_test_output.predictions[-1])
wandb.save(baseline_answers_path);
wandb.finish()

train/loss,14.5045
train/learning_rate,0.0
train/epoch,5.0
train/f1,0.0
train/accuracy,0.0
train/em,0.0
_step,55
_runtime,142
_timestamp,1611082617
eval/loss,14.63105
eval/f1,0.00016


train/loss,▂▇█▄▇▆▃▆▂▃▃▁
train/learning_rate,█▇▇▆▆▅▄▄▃▂▂▁
train/epoch,▁▂▂▂▃▃▄▄▅▅▅▆▇▇▇█
train/f1,▄▁▂▁▁▁▅▁▂█▁
train/accuracy,▆▃▄▁▃▁█▁▆▆▁
train/em,▁▁▁▁▁▁█▁▁▁▁
_step,▁▂▂▂▃▃▄▄▅▅▅▆▇▇▇█
_runtime,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃█
_timestamp,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃█
eval/loss,█▆▄▂▁
eval/f1,▁▁▁▁▁


## BiDAF

In [39]:
bidaf_model = model.QABiDAFModel(glove_embedding_layer, device=DEVICE)
print(f"The BiDAF model has {bidaf_model.count_parameters()} parameters")

The BiDAF model has 79675 parameters


In [40]:
bidaf_optimizer = optim.Adadelta(bidaf_model.parameters(), lr=0.5)
bidaf_lr_scheduler = optim.lr_scheduler.ExponentialLR(bidaf_optimizer, gamma=.999)

In [41]:
%env WANDB_RUN_GROUP=bidaf
bidaf_run_name=utils.get_run_name()
bidaf_output_dir=f"./checkpoints/bidaf/{bidaf_run_name}"
bidaf_args = TRAINER_ARGS(output_dir=bidaf_output_dir,run_name=bidaf_run_name)
bidaf_trainer = training.SquadTrainer(
    model=bidaf_model,
    args=bidaf_args,
    data_collator=standard_dm.tokenizer,
    train_dataset=standard_dm.train_dataset,
    eval_dataset=standard_dm.val_dataset,
    optimizers=(bidaf_optimizer, bidaf_lr_scheduler),
)

env: WANDB_RUN_GROUP=bidaf


In [42]:
bidaf_trainer.train()


  0%|          | 0/55 [00:00<?, ?it/s]


  2%|▏         | 1/55 [00:02<02:33,  2.84s/it]


Exception: The wandb backend process has shutdown

In [ ]:
bidaf_test_output = bidaf_trainer.predict(standard_dm.test_dataset)
bidaf_test_output.metrics

In [45]:
bidaf_answers_path = "results/bidaf.json"
utils.save_answers(bidaf_answers_path, bidaf_test_output.predictions[-1])
wandb.save(bidaf_answers_path);
wandb.finish()

Error: You must call wandb.init() before wandb.save()

## BERT

In [87]:
MAX_BERT_TOKENS = 512

In [47]:
bert_model = model.QABertModel(device=DEVICE)

In [48]:
bert_wp_tokenizer = BertWordPieceTokenizer("data/bert-base-uncased-vocab.txt", lowercase=True)
bert_wp_tokenizer.enable_padding(
    direction="right",
    pad_type_id=1,
)
bert_wp_tokenizer.enable_truncation(MAX_BERT_TOKENS)

NameError: name 'MAX_BERT_TOKENS' is not defined

In [ ]:
bert_tokenizer = dataset.BertSquadTokenizer(bert_wp_tokenizer, device=DEVICE)

In [ ]:
bert_dm = dataset.SquadDataManager(squad_dataset, bert_tokenizer, device=DEVICE)

In [ ]:
bert_dm.tokenizer = bert_tokenizer

In [ ]:
bert_dm.train_df

In [ ]:
bert_dm.val_df

In [ ]:
bert_dm.test_df

In [55]:
%env WANDB_RUN_GROUP=bert
bert_run_name=utils.get_run_name()
bert_output_dir=f"./checkpoints/bert/{bert_run_name}"
bert_args = TRAINER_ARGS(output_dir=bert_output_dir,run_name=bert_run_name, per_device_train_batch_size=4, per_device_eval_batch_size=4,)
bert_trainer = training.SquadTrainer(
    model=bert_model,
    args=bert_args,
    data_collator=bert_dm.tokenizer,
    train_dataset=bert_dm.train_dataset,
    eval_dataset=bert_dm.val_dataset,
)

env: WANDB_RUN_GROUP=bert


In [100]:
bert_trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy,Em,Runtime,Samples Per Second
1,14.225100,17.034660,0.001423,0.008671,0.000000,3.991000,43.347000
2,14.787900,17.034660,0.001423,0.008671,0.000000,4.000600,43.243000


wandb: WARNING Step must only increase in log calls.  Step 1 < 175; dropping {'train/loss': 13.748, 'train/learning_rate': 0.0, 'train/epoch': 0.01}.
wandb: WARNING Step must only increase in log calls.  Step 1 < 175; dropping {'train/f1': 0.0, 'train/accuracy': 0.0, 'train/em': 0.0, 'train/epoch': 0.01}.
wandb: WARNING Step must only increase in log calls.  Step 5 < 175; dropping {'train/loss': 12.9371, 'train/learning_rate': 0.0, 'train/epoch': 0.03}.
wandb: WARNING Step must only increase in log calls.  Step 5 < 175; dropping {'train/f1': 0.0, 'train/accuracy': 0.0, 'train/em': 0.0, 'train/epoch': 0.03}.
wandb: WARNING Step must only increase in log calls.  Step 10 < 175; dropping {'train/loss': 14.0872, 'train/learning_rate': 0.0, 'train/epoch': 0.06}.
wandb: WARNING Step must only increase in log calls.  Step 10 < 175; dropping {'train/f1': 0.0, 'train/accuracy': 0.0, 'train/em': 0.0, 'train/epoch': 0.06}.
wandb: WARNING Step must only increase in log calls.  Step 15 < 175; droppi

wandb: WARNING Step must only increase in log calls.  Step 130 < 175; dropping {'train/f1': 0.0, 'train/accuracy': 0.0, 'train/em': 0.0, 'train/epoch': 0.74}.
wandb: WARNING Step must only increase in log calls.  Step 135 < 175; dropping {'train/loss': 13.0339, 'train/learning_rate': 0.0, 'train/epoch': 0.77}.
wandb: WARNING Step must only increase in log calls.  Step 135 < 175; dropping {'train/f1': 0.0, 'train/accuracy': 0.0, 'train/em': 0.0, 'train/epoch': 0.77}.
wandb: WARNING Step must only increase in log calls.  Step 140 < 175; dropping {'train/loss': 13.8517, 'train/learning_rate': 0.0, 'train/epoch': 0.8}.
wandb: WARNING Step must only increase in log calls.  Step 140 < 175; dropping {'train/f1': 0.0, 'train/accuracy': 0.0, 'train/em': 0.0, 'train/epoch': 0.8}.
wandb: WARNING Step must only increase in log calls.  Step 145 < 175; dropping {'train/loss': 15.33, 'train/learning_rate': 0.0, 'train/epoch': 0.83}.
wandb: WARNING Step must only increase in log calls.  Step 145 < 175

In [ ]:
bert_test_output = bert_trainer.predict(bert_dm.test_dataset)
bert_test_output.metrics

In [ ]:
bert_answers_path = "results/bert.json"
utils.save_answers(bert_answers_path, bert_test_output.predictions[-1])
wandb.save(bert_answers_path);
wandb.finish()